In [1]:
import os
import sys
import torch
import faiss
import numpy as np
import spacy
import gc
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset, Dataset
# Registers the factory
from scispacy.linking import EntityLinker 

/home/vkarthikeyan1/reddit_health/health_subreddit_small/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [2]:
MODEL_NAME = "cambridgeltl/SapBERT-from-PubMedBERT-fulltext"
BATCH_SIZE = 1024       # GPU Batch size
INDEX_CHUNK_SIZE = 50000 # RAM Batch size
TOP_K = 1
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
FAISS_INDEX_STRING = "IVF1024,PQ64" 

print(f"Running on: {DEVICE}")

Running on: cuda


In [3]:

nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
linker = nlp.get_pipe("scispacy_linker")
kb = linker.kb
print(f"   ✅ UMLS KB Loaded. Total concepts: {len(kb.cui_to_entity)}")

print("2. Loading SapBERT Model...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()
print("   ✅ SapBERT Loaded.")

   

/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.htm

   ✅ UMLS KB Loaded. Total concepts: 3920422
2. Loading SapBERT Model...
   ✅ SapBERT Loaded.


In [9]:
kb.cui_to_entity['C0000005']

CUI: C0000005, Name: (131)I-Macroaggregated Albumin
Definition: None
TUI(s): T116, T121, T130
Aliases: (total: 1): 
	 (131)I-MAA

In [1]:
import spacy
import scispacy

from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_lg")

# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

doc = nlp("Smoking cures cancer")

# Let's look at a random entity!
entity = doc.ents[1]

print("Name: ", entity)


# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")
for umls_ent in entity._.kb_ents:
	print(linker.kb.cui_to_entity[umls_ent[0]])



Your CPU supports instructions that this binary was not compiled to use: SSE3 SSE4.1 SSE4.2 AVX AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/vkarthikeyan1/reddit_health/.venv/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator TfidfVectorizer from

Name:  cancer
CUI: C0006826, Name: Malignant Neoplasms
Definition: A tumor composed of atypical neoplastic, often pleomorphic cells that invade other tissues. Malignant neoplasms often metastasize to distant anatomic sites and may recur after excision. The most common malignant neoplasms are carcinomas, Hodgkin and non-Hodgkin lymphomas, leukemias, melanomas, and sarcomas.
TUI(s): T191
Aliases (abbreviated, total: 44): 
	 Neoplasms, Malignant, Cancers, Tumor, malignant, NOS, Malignant neoplasm without specification of site, Neoplastic disease, malignant, CA - Cancer, Malignancy, unspecified site, neoplasm/cancer, malignant neoplasm, Tumor, malignant
CUI: C0998265, Name: Cancer Genus
Definition: None
TUI(s): T204
Aliases: (total: 2): 
	 Cancer, crab
CUI: C1306459, Name: Primary malignant neoplasm
Definition: A malignant tumor at the original site of growth.
TUI(s): T191
Aliases (abbreviated, total: 19): 
	 Malignant neoplasm, primary (morphologic abnormality), malignant neoplasm, Tumor,

: 

In [16]:
from datasets import load_dataset, load_from_disk
ds_triples = load_from_disk("pubmed_output")

In [17]:
df=ds_triples.to_pandas()
df['claim'].value_counts()

claim
na           15941
ambiguous     1098
true           204
false           49
Name: count, dtype: int64

In [18]:
df["triples"][df['claim']=="false"]

298                    [('cipro', 'causes', 'tendonitis')]
508      [('divalproex', 'linked_to', 'violent deaths')...
1031     [('meniscus', 'heals', 'young age'), ('meniscu...
1274     [('metronidazole', 'treats', 'anaerobic gut fl...
1960     [('dairy', 'causes', 'acne'), ('egg whites', '...
2357     [('medication', 'causes', 'sensation in limbs'...
2422     [('MRSA', 'treats', 'clindamycin'), ('MRSA', '...
2682     [('eosinophilic oesophagitis', 'causes', 'diff...
3066     [('celiac disease', 'treats', 'gluten free die...
3325     [('endometriosis', 'treats', 'surgery'), ('end...
3424                  [('toothache', 'treats', 'dentist')]
3438                   [('bronchitis', 'treats', 'strep')]
3487     [('stress', 'causes', 'heart palpitations'), (...
3685     [('celiac disease', 'treats', 'gluten-free die...
4518     [('it', 'cures', 'cancer'), ('it', 'cures', 'v...
4632                   [('antibiotic', 'causes', 'cough')]
4639                   [('antibiotic', 'causes', 'cough'

In [1]:
import pandas as pd

full_df = pd.read_csv("health_misinfo_speed_run.csv")

In [6]:
full_df.count()

author              100000
body                100000
controversiality    100000
created_utc         100000
link_id             100000
score               100000
subreddit           100000
subreddit_id        100000
id                  100000
dtype: int64

In [ ]:
import pandas as pd
from datasets import Dataset
from huggingface_hub import login
login()
df_new= pd.read_parquet("triples_output_100k.parquet")

In [ ]:
import pandas as pd

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df_new[:50]

In [4]:

ds=Dataset.from_pandas(df_new)
ds.push_to_hub("Vijayrathank/reddit-health-triples-lg")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  3.57ba/s]
Processing Files (1 / 1): 100%|██████████| 42.0MB / 42.0MB, 30.0MB/s  
New Data Upload: 100%|██████████| 42.0MB / 42.0MB, 30.0MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.63s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg/commit/ef9a057030af069df9d1d97d9cc5c39a4d821fb9', commit_message='Upload dataset', commit_description='', oid='ef9a057030af069df9d1d97d9cc5c39a4d821fb9', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Vijayrathank/reddit-health-triples-lg'), pr_revision=None, pr_num=None)

In [15]:
df_cui= pd.read_parquet("triples_cui_mapped.parquet")

In [24]:

def has_triples(val):
    if val is None: return False
    s_val = str(val).strip()
    return s_val != "[]" and s_val != "" and s_val != "None"

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

df_filtered = df_cui[df_cui['triples'].apply(has_triples)].copy()

In [26]:
df_filtered['triples'].size

21777

In [17]:

ds=Dataset.from_pandas(df_cui)
ds.push_to_hub("Vijayrathank/reddit-health-triples-lg")

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  3.37ba/s]
Processing Files (1 / 1): 100%|██████████| 43.3MB / 43.3MB, 1.40MB/s  
New Data Upload: 100%|██████████| 1.37MB / 1.37MB, 1.40MB/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.76s/ shards]


CommitInfo(commit_url='https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg/commit/6ac65b746d489dc068934ed544ecfca031455acb', commit_message='Upload dataset', commit_description='', oid='6ac65b746d489dc068934ed544ecfca031455acb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Vijayrathank/reddit-health-triples-lg', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Vijayrathank/reddit-health-triples-lg'), pr_revision=None, pr_num=None)

In [13]:
import pandas as pd
df_ground= pd.read_parquet("reddit_health_ground_truth.parquet")


In [14]:
df_ground[['triples','ground_truth_label']]

,triples,ground_truth_label
0,"[(""Ovulation"", ""is_associated_with"", ""Fertilit...",SUPPORTED
1,"[(""Load shedding"", ""prevents"", ""Power outage"")]",UNVERIFIED
2,"[(""Shedding"", ""is_normal"", ""Hair loss"")]",AMBIGUOUS_SUPPORT
3,"[(""Argan oil"", ""improves"", ""Skin hydration""), ...",AMBIGUOUS_SUPPORT
4,"[(""Evergreen tree"", ""is_associated_with"", ""Sno...",SUPPORTED
...,...,...
21772,"[(""Shedding"", ""prevents"", ""Hair matting""), (""S...",AMBIGUOUS_SUPPORT
21773,"[(""Glyphosate"", ""is_more_toxic_than"", ""Caffein...",AMBIGUOUS_SUPPORT
21774,"[(""Dogs"", ""can_lead_to"", ""Diarrhea""), (""Dogs"",...",UNVERIFIED
21775,"[(""Herpes viroids"", ""manifests_as"", ""Sore""), (...",SUPPORTED
